In [5]:
import psycopg2
from decouple import config
import pandas as pd
# make sure to add .env file with the credentials below in this same directory for this notebook to work
%load_ext dotenv
%dotenv

In [7]:
with psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
) as psql_conn:
    
    curs = psql_conn.cursor()
    curs.execute("""
        SELECT COUNT(*)
        FROM job_listings
    """)
    print(curs.fetchone())

(172280,)


In [8]:
conn = psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
)
df = pd.read_sql("""
    SELECT *
    FROM job_listings;
""", conn)



In [ ]:
with open("job_listings.csv", "w") as f:
    df.to_csv(f)

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
import re

def tokenize(doc):
    return [
        re.sub(r"[^a-z0-9]", "", t.lemma_.lower()).strip() for t in tokenizer(doc)
        if not t.is_stop and not t.is_punct and t.text.strip()
    ]

df["tokens"] = df["title"].apply(tokenize)
df.head()

In [ ]:
df["is_nurse"] = df["tokens"].apply(lambda x: "nurse" in x)

In [ ]:
df["is_nurse"].sum()

In [ ]:
nurse_rows = df[df["is_nurse"]].index

In [ ]:
conn = psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
)
curs = conn.cursor()

curs.execute("""
    SELECT *
    FROM lda17_topics
""")

In [ ]:
lda_df = pd.read_sql("""
    SELECT * FROM lda17_topics
""", conn)
lda_df.head()

In [ ]:
curs = conn.cursor()
execute_batch(curs, """
    DELETE FROM lda17_topics
    WHERE job_id=%s;
""", [[ix] for ix in nurse_rows])
execute_batch(curs, """
    DELETE FROM job_listings
    WHERE id=%s;
""", [[ix] for ix in nurse_rows])

curs.close()
conn.commit()

In [ ]:
curs = conn.cursor()

curs.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public';
""")
curs.fetchall()

In [ ]:

df = pd.read_sql("""
    SELECT *
    FROM job_listings;
""", conn)

descriptions_df = pd.read_sql("""
    SELECT *
    FROM job_descriptions;
""", conn)
companies_df = pd.read_sql("""
    SELECT *
    FROM job_companies;
""", conn)
keyphrases_df = pd.read_sql("""
    SELECT *
    FROM job_keyphrases;
""", conn)
locations_df = pd.read_sql("""
    SELECT *
    FROM job_locations;
""", conn)
descriptions_df.head()

In [ ]:
companies_df.head()

In [ ]:
keyphrases_df.head()

In [ ]:
locations_df.head()

In [ ]:
df = df.merge(descriptions_df, left_on="id", right_on="job_id", how="outer")

In [ ]:
df = df.merge(companies_df, left_on="job_id", right_on="job_id", how="outer")
df = df.merge(keyphrases_df, left_on="job_id", right_on="job_id", how="outer")
df = df.merge(locations_df, left_on="job_id", right_on="job_id", how="outer")

In [ ]:
for col in df.columns:
    print(f"\"{col}\"")

In [ ]:
df = df.drop(columns=["id", "id_x", "id_y"])
df.head()

In [ ]:
companies_df2 = pd.read_sql("""
    SELECT *
    FROM companies;
""", conn)

In [ ]:
df = df.merge(companies_df2, left_on="company_id", right_on="id", how="outer")

In [ ]:
df.head()

In [ ]:
locations_df2 = pd.read_sql("""
    SELECT *
    FROM locations;
""", conn)
df = df.merge(locations_df2, left_on="company_id", right_on="id", how="outer")
df.head()

In [ ]:
df = df.drop(columns=["id_x", "id_y"])


In [ ]:
df = df.rename(
    columns={
        "description_x": "job_description",
        "description_y": "company_description",
    }
)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
with open("job_listings.csv", "w") as f:
    df.to_csv(f)

In [ ]:
df["job_description"].str.replace(r"\s+", " ")